# Deep Learning and Hybrid Models for Electricity Consumption Prediction

In this notebook, we implement various deep learning models for residential building electricity consumption forecasting based on the provided datasets.<br>The models considered include LSTM, Bi-LSTM, GRU, Bi-GRU, 1D-CNN, and TCN.<br>In addition, we explore hybrid models such as LSTM-TCN, BiLSTM-TCN, GRU-TCN, and BiGRU-TCN.

The hyperparameters are set as per your specifications:
* Optimizer: Adam
* Learning rate: 0.001
* Attention mechanism: True
* Activation function: LeakyReLU
* Random state: 42
* Batch size: 24
* Epochs: 100

The evaluation metrics used are as follows:
* Mean absolute percentage error (MAPE);
* Coefficient of variation of the root mean squared error (CVRMSE);
* Normalized mean absolute error (NMAE).

In [ ]:
# Install necessary libraries
!pip install keras-tcn keras-self-attention

# Import essential libraries
import time
import datetime
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Set random seed for reproducibility
np.random.seed(42)
import tensorflow as tf
tf.random.set_seed(42)

# Import deep learning libraries
from tensorflow import keras
from tensorflow.keras.layers import (TimeDistributed, Bidirectional, GRU, LSTM,
                                     Conv1D, Flatten, Dense, Concatenate, RepeatVector,
                                     GlobalAveragePooling1D, MaxPooling1D, Reshape, Activation)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from keras_self_attention import SeqSelfAttention
from tcn import TCN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Suppress TensorFlow warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

## Load and Split the Datasets

We use the provided `load_and_split_data' function to load either the 'household' or 'dormitory' dataset.<br>The data are split into training and test sets based on predefined indices.

In [ ]:
def load_and_split_data(dataset_choice):
    if dataset_choice == 'household':
        data = pd.read_csv('Appliances Energy Prediction.csv')
        X_train = data.iloc[:2311, 1:-1]
        y_train = data.iloc[:2311, -1]
        X_test = data.iloc[2311:, 1:-1]
        y_test = data.iloc[2311:, -1]
    elif dataset_choice == 'dormitory':
        data = pd.read_csv('University Residential Complex.csv')
        X_train = data.iloc[:20472, 5:-1]
        y_train = data.iloc[:20472, -1]
        X_test = data.iloc[20472:, 5:-1]
        y_test = data.iloc[20472:, -1]
    else:
        raise ValueError("Invalid dataset choice. Please select 'household' or 'dormitory'.")
    return X_train, X_test, y_train, y_test

# Select the dataset
dataset_choice = 'household'  # Change to 'dormitory' as needed
X_train_raw, X_test_raw, y_train_raw, y_test_raw = load_and_split_data(dataset_choice)

# Display shapes
print("Training set shape:", X_train_raw.shape, y_train_raw.shape)
print("Test set shape:", X_test_raw.shape, y_test_raw.shape)

## Data Preprocessing

We scale the features using `MinMaxScaler` to standardize the data.<br>We then prepare the data for time series forecasting by creating sequences with `n_past` time steps to predict `n_future` time steps.

In [ ]:
# Scaling the data
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train_raw)
X_test_scaled = scaler_X.transform(X_test_raw)
y_train_scaled = scaler_y.fit_transform(y_train_raw.values.reshape(-1, 1))
y_test_scaled = scaler_y.transform(y_test_raw.values.reshape(-1, 1))

# Define past and future time steps
n_past = 24   # Number of past time steps
n_future = 24  # Number of future time steps to predict
y_feature = 1  # Since we're predicting one feature (energy consumption)

# Prepare data for time series forecasting
def create_sequences(X, y, n_past, n_future):
    Xs, ys = [], []
    for i in range(n_past, len(X) - n_future +1):
        Xs.append(X[i - n_past:i])
        ys.append(y[i:i + n_future])
    return np.array(Xs), np.array(ys)

# Create sequences for training and testing
X_train_seq, y_train_seq = create_sequences(X_train_scaled, y_train_scaled, n_past, n_future)
X_test_seq, y_test_seq = create_sequences(X_test_scaled, y_test_scaled, n_past, n_future)

print("Training sequences shape:", X_train_seq.shape, y_train_seq.shape)
print("Testing sequences shape:", X_test_seq.shape, y_test_seq.shape)

## Define Evaluation Metrics

We define functions for MAPE, CVRMSE, and NMAE.

In [ ]:
def MAPE(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = y_true != 0
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100

def CVRMSE(y_true, y_pred):
    return (np.sqrt(np.mean((y_true - y_pred) ** 2)) / np.mean(y_true)) * 100

def NMAE(y_true, y_pred):
    return (np.mean(np.abs(y_true - y_pred)) / np.mean(y_true)) * 100

## Build and Train Models

We define a function to build and train various deep learning models based on the selected architecture.<br>The models include LSTM, Bi-LSTM, GRU, Bi-GRU, 1D-CNN, TCN, and hybrid models such as LSTM-TCN, BiLSTM-TCN, GRU-TCN, and BiGRU-TCN.<br>We use the LeakyReLU activation function and standardize the hyperparameters as specified.

In [ ]:
def model_training(model_select, attention, activation):
    encoder_inputs = Input(shape=(n_past, X_train_seq.shape[2]))
    y_feature = 1  # Number of features to predict

    if model_select == 'lstm-tcn':
        # LSTM-TCN Hybrid Model
        encoder_lstm = LSTM(14, return_state=True, activation='linear')
        encoder_outputs1 = encoder_lstm(encoder_inputs)
        encoder_states1 = encoder_outputs1[1:]  # Hidden and cell states
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_tcn = TCN(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_tcn = LeakyReLU()(decoder_tcn)
        decoder_l1 = LSTM(14, return_sequences=True, activation='linear')(decoder_tcn, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)

    elif model_select == 'bilstm-tcn':
        # BiLSTM-TCN Hybrid Model
        encoder_bilstm = Bidirectional(LSTM(14, return_state=True, activation='linear'))
        encoder_outputs1 = encoder_bilstm(encoder_inputs)
        encoder_states1 = encoder_outputs1[1:]
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_tcn = TCN(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_tcn = LeakyReLU()(decoder_tcn)
        decoder_l1 = Bidirectional(LSTM(14, return_sequences=True, activation='linear'))(decoder_tcn, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)

    elif model_select == 'gru-tcn':
        # GRU-TCN Hybrid Model
        encoder_gru = GRU(14, return_state=True, activation='linear')
        encoder_outputs1 = encoder_gru(encoder_inputs)
        encoder_states1 = [encoder_outputs1[1]]  # Hidden state
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_tcn = TCN(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_tcn = LeakyReLU()(decoder_tcn)
        decoder_l1 = GRU(14, return_sequences=True, activation='linear')(decoder_tcn, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)

    elif model_select == 'bigru-tcn':
        # BiGRU-TCN Hybrid Model
        encoder_bigru = Bidirectional(GRU(14, return_state=True, activation='linear'))
        encoder_outputs1 = encoder_bigru(encoder_inputs)
        encoder_states1 = [encoder_outputs1[1]]  # Hidden state
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_tcn = TCN(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_tcn = LeakyReLU()(decoder_tcn)
        decoder_l1 = Bidirectional(GRU(14, return_sequences=True, activation='linear'))(decoder_tcn, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == 'lstm':
        # LSTM Single Model
        encoder_lstm = LSTM(14, return_state=True, activation='linear')
        encoder_outputs1 = encoder_lstm(encoder_inputs)
        encoder_states1 = encoder_outputs1[1:]
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_l1 = LSTM(14, return_sequences=True, activation='linear')(decoder_inputs, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == 'bilstm':
        # BiLSTM Single Model
        encoder_bilstm = Bidirectional(LSTM(14, return_state=True, activation='linear'))
        encoder_outputs1 = encoder_bilstm(encoder_inputs)
        encoder_states1 = encoder_outputs1[1:]
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_l1 = Bidirectional(LSTM(14, return_sequences=True, activation='linear'))(decoder_inputs, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == 'gru':
        # GRU Single Model
        encoder_gru = GRU(14, return_state=True, activation='linear')
        encoder_outputs1 = encoder_gru(encoder_inputs)
        encoder_states1 = [encoder_outputs1[1]]
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_l1 = GRU(14, return_sequences=True, activation='linear')(decoder_inputs, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == 'bigru':
        # BiGRU Single Model
        encoder_bigru = Bidirectional(GRU(14, return_state=True, activation='linear'))
        encoder_outputs1 = encoder_bigru(encoder_inputs)
        encoder_states1 = [encoder_outputs1[1]]
        encoder_outputs = LeakyReLU()(encoder_outputs1[0])
        decoder_inputs = RepeatVector(n_future)(encoder_outputs)
        decoder_l1 = Bidirectional(GRU(14, return_sequences=True, activation='linear'))(decoder_inputs, initial_state=encoder_states1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == '1d-cnn':
        # 1D CNN Model
        conv1d = Conv1D(filters=14, kernel_size=2, activation='linear')(encoder_inputs)
        conv1d = LeakyReLU()(conv1d)
        flatten = Flatten()(conv1d)
        decoder_inputs = RepeatVector(n_future)(flatten)
        decoder_l1 = LSTM(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    elif model_select == 'tcn':
        # TCN Model
        tcn_layer = TCN(14, return_sequences=False, activation='linear')(encoder_inputs)
        tcn_layer = LeakyReLU()(tcn_layer)
        decoder_inputs = RepeatVector(n_future)(tcn_layer)
        decoder_l1 = TCN(14, return_sequences=True, activation='linear')(decoder_inputs)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    else:
        raise ValueError("Invalid model selection.")

    # Attention mechanism
    if attention:
        decoder_l1 = SeqSelfAttention(attention_activation='linear')(decoder_l1)
        decoder_l1 = LeakyReLU()(decoder_l1)
    
    decoder_outputs1 = TimeDistributed(Dense(y_feature, activation='linear'))(decoder_l1)
    model_result = Model(encoder_inputs, decoder_outputs1)
    model_result.summary()
    return model_result

### Training the Models

Each model will be trained for 100 epochs with a batch size of 24, aligning with the daily power consumption cycle.<br>We use the `EarlyStopping` callback to prevent overfitting.<br>A random state of 42 ensures reproducibility.

The hyperparameters are standardized across experiments as specified:
- Optimizer: Adam with a learning rate of 0.001
- Activation function: LeakyReLU
- Attention mechanism: True
- Batch size: 24
- Epochs: 100
- Random state: 42

In [ ]:
# Define the optimizer
optimizer = Adam(learning_rate=0.001)

# Prepare callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

# List of models to train
model_list = ['lstm', 'bilstm', 'gru', 'bigru', '1d-cnn', 'tcn', 'lstm-tcn', 'bilstm-tcn', 'gru-tcn', 'bigru-tcn']

# Dictionary to store trained models
results = {}

# Training and evaluation loop
for model_name in model_list:
    print(f"\nTraining model: {model_name}")
    # Build the model
    model = model_training(model_select=model_name, attention=True, activation='leaky_relu')
    # Compile the model
    model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
    # Train the model
    history = model.fit(
        X_train_seq, y_train_seq,
        epochs=100,
        batch_size=24,
        validation_split=0.1,
        callbacks=[early_stop, tensorboard_callback],
        verbose=1
    )
    # Save the trained model
    results[model_name] = model

## Evaluate Models

We evaluate each model using the defined metrics and compare their performance.<br>We also save the evaluation metrics in a CSV file named `deep_learning_evaluation_metrics.csv`.

In [ ]:
# Initialize a list to store evaluation results
evaluation_results = []

for model_name in model_list:
    print(f"\nEvaluating model: {model_name}")
    model = results[model_name]
    # Predict on the test set
    y_pred_scaled = model.predict(X_test_seq)
    # Reshape predictions and true values
    y_pred_scaled = y_pred_scaled.reshape(-1, 1)
    y_test_scaled_flat = y_test_seq.reshape(-1, 1)
    # Inverse transform to get actual values
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    y_true = scaler_y.inverse_transform(y_test_scaled_flat)
    # Calculate evaluation metrics
    mape = MAPE(y_true, y_pred)
    cvrmse = CVRMSE(y_true, y_pred)
    nmae = NMAE(y_true, y_pred)
    # Print evaluation results
    print(f"Evaluation results for {model_name}:")
    print(f"MAPE: {mape:.2f}%")
    print(f"CVRMSE: {cvrmse:.2f}%")
    print(f"NMAE: {nmae:.2f}%")
    # Append results to evaluation_results
    evaluation_results.append({
        'Model': model_name,
        'MAPE': mape,
        'CVRMSE': cvrmse,
        'NMAE': nmae
    })

# Save evaluation metrics to a .csv file
csv_file_name = "deep_learning_evaluation_metrics.csv"
with open(csv_file_name, "w", newline="") as csvfile:
    fieldnames = ["Model", "MAPE", "CVRMSE", "NMAE"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for result in evaluation_results:
        writer.writerow(result)

## Conclusions

We trained and evaluated several deep learning models, including single and hybrid models, for residential building electricity consumption forecasting.<br>The models were implemented exactly as specified in the provided code and trained with standardized hyperparameters to ensure a fair comparison.<br>The evaluation metrics indicate the performance of the models and we can select the best performing model based on these results.